Source: https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf

One of the first papers that presented details on NN approach to recommendations

Key innovations:
- Two-stage recommendation process: 
    1. candidate generation (fast but crude) 
    2. candidate ranking (slow but precise)

What benefits does two-stage process provide:
- recall - allows efficient navigation through large item space
- precision - picks relevant and engaging items for user
- blending - allows using using multiple sources

<img src="img/youtube3.png" width=500>

### Candidate Generation Network
Candidate network is a DNN that combines multiple user features into single embedding. 

*User features are very dynamic since they include history => probably "query features" would be a better name

This network implements collaborative filering in a sense that it does not rely on content but rely on averaged user watching patterns (those patterns are encoded in embedding space)

Model specification
- input = user data (user watch history + user query history + user socdem)
- pre-output = user embedding
- output = softmaxed user embedding multipled by all video vectors
- loss function = cross-entropy of the next video prediction

<img src="img/youtube1.png" width=750>

At training time DNN learns query embeddings in a way that it can easily predict next video by taking dot product with all possible video vectors (see "softmax => class probabilities")

At inference time DNN performs learned processing and picks top-N most similar videos from a video storage

#### Important implementation details

- watch vector consists of 50 recent video watches
- search vector consists of 50 recent searches
- each item and search are encoded with 256 vector embedding
- recent searches and watches are not ordered (bag-of-videos / bag-of-searches)
- video Vocabulary = 1M video embeddings
- search Vocabulary = 1M search string embeddings

Negative sampling

### Candidate Ranking Network

Ranker is a pointwise scoring network trained to predict click probability (optionally weighted by watch time of the click)

It uses all user features as well as a lot more video features (as well as user-video interaction features)

<img src="img/youtube2.png" width=750>

# Дайджест работы

https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf

Deep Learning драматически улучшил результаты

Challenges:
- scale
- variability
- noise (?)

Built on rails of Tensorflow, 1B model, 100B training samples

До этого DL пытались применять, но не очень часто. Несколько примеров:
- AutoRec (2015)<br>вектора рейтингов восстанавливаются обычным автоенкодером
- Collaborative DL (2015)<br>комбинируют контентные фичи с рейтингами (зачем-то в генеративной постановке), которые кодируют глубоким автоенкодером
- Deep Music (2013)<br>одна из первых контентных систем для музыки


### Generation

Смотрят только на implicit сигнал, так как его в разы больше




Задача мультиклассификации на 1M классов (Extreme Multiclass classification)
- сэмплирование 1000 негативов
- кросс-энтропия?
- пробовали еще иерархический софтмакс, но не залетел

Кандидаты отбираются с помощью ANN для поиска соседей. Метод не описывается (что-то с хэшированием), но отмечается, что выбор конкретной модели сильно не влияет на качество

В качестве фичи добавляют возраст видео (example age), так как считается, что "свежесть" контента влияет на потребление

### Ranking

что делает = оценивает каждого кандидата на показ из набранного пула с точки зрения, насколько подходит пользователю

чем полезен:
- мало кандидатов => можно точнее персонализировать
- часто рекомендации из разных источников несравнимы => ранжирует уже финальный список

целевая фунцкия = оптимизируется под E[watch_time] (у отсутствия клика его считаем отрицательным)<br>
*Можно и под CTR, но он подвержен кликбейтовости

фичей много (несколько сотен). Среди них есть:
- категориальные / непрерывные
- бинарные (факт залогина) / высококардинальные (последний запрос)
- univalent / multivalent (список просмотров)

по типу:
- impression features = фичи оцениваемого видео
- query features = фичи user + context<br>считается только раз на запрос
- user-video interaction features = отношение с пользователем <br>например, similar videos rating, topic, channel<br>важны, так как
    - легче обобщать пользовательские предпочтения
    - допускают 'затухание' неинтересных рекомендаций

детали:
- категориальные фичи кодируются словарями эмбедингов<br>если значений много, оставляют только некий фиксированный топ
- если фича входит несоклько раз, словарь используется один <br>shared embedding
- непрерывные фичи нормализуются до интревала [0,1]
- источник рекомендации и скор тоже важные фичи на этапе ранжирования
